<a href="https://colab.research.google.com/github/Somertonman/hackathon_2023/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import pandas as pd
import warnings
from tqdm import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None 

In [ ]:
!pip install pymorphy2
import pymorphy2

In [ ]:
!pip install transformers sacremoses
from transformers import pipeline

qa_pipeline = pipeline("question-answering", "timpal0l/mdeberta-v3-base-squad2")

In [ ]:
!wget -O dataset https://www.dropbox.com/sh/d5h5f3yrql8x392/AACQ2WYa5qYCqjC8QuVZ5TJ4a?dl=1

In [ ]:
!unzip dataset

In [ ]:
!tar -xvf  '/content/task_3/full_dataset/json.tar' -C '/content/task_3'

In [97]:
def im_pad(text):
  morph = pymorphy2.MorphAnalyzer()
  words = text.split()
  nominative_words = []
  for word in words:
    parsed_word = morph.parse(word)[0]
    nominative_word = parsed_word.normal_form
    nominative_words.append(nominative_word)
  nominative_sentence = " ".join(nominative_words)
  return nominative_sentence

In [125]:
def parse_file(file_name, num):
  df = pd.read_json(file_name).reset_index()
  
  df['id'] = num

  df = df.query('index not in ("Global_deadline", "Global_supervisor")')
  df['task_text'] = df.apply(lambda x: x['Tasks']['task_text'], axis = 1)
  df['task_responsibles_people'] = df.apply(lambda x: x['Tasks']['task_responsibles_people'], axis = 1)
  df['task_responsibles_groups'] = df.apply(lambda x: x['Tasks']['task_responsibles_groups'], axis = 1)

  question = "Какая должность?"
  df['position'] = df.apply(lambda x: im_pad(qa_pipeline(question, x['task_responsibles_people'])['answer']), axis = 1)

  question = "Какое имя?"
  df['name'] = df.apply(lambda x: im_pad(qa_pipeline(question, x['task_responsibles_people'])['answer']), axis = 1)

  df = df[['id', 'index', 'task_text', 'task_responsibles_groups', 'position', 'name']]
  return df.rename(columns={'index':'task_num', 'task_responsibles_groups':'dept'})

In [130]:
df = parse_file('/content/task_3/json/0.json', 0)

for i in tqdm(range (1, 10)):
  file_name = f"/content/task_3/json/{i}.json"
  df_one = parse_file(file_name, i)
  df = pd.concat([df, df_one])

df = df.query('name != "себя"').reset_index().drop(columns='index')

100%|██████████| 9/9 [00:27<00:00,  3.05s/it]


In [127]:
df

,id,task_num,task_text,dept,position,name
2,0,Task1,Закупить книги. Срок завершения: 19 апр 15.,Административно-хозяйственный отдел,специалист первый категория административно-хо...,кравцов и.е.
